In [1]:
import os

import pandas as pd

In [2]:
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.width", 300)

In [3]:
tol = 1e-6

In [4]:
bench_opf_dir = "../bench/opf"
bench_res_path = "../bench/opf/matpower-pip_acopf_benchmark_results.csv"
df_bench_res = pd.read_csv(bench_res_path)
df_bench_res_success = df_bench_res[
    df_bench_res["matpower-pip_acopf_status"] == "Success"
]
for file_name in df_bench_res_success["file_name"]:
    file_name_no_ext = file_name.split(".")[0]  # remove ext
    file_path = os.path.join(bench_opf_dir, file_name_no_ext + ".gen.csv")
    df_gen = pd.read_csv(file_path)
    cons_pg = df_gen["PG"] > df_gen["PMAX"] + tol
    if any(cons_pg):
        print(os.path.basename(file_path))

In [5]:
case_name = "pglib_opf_case2746wop_k.gen.csv"
file_path = os.path.join(bench_opf_dir, case_name)
df_gen = pd.read_csv(file_path, index_col=0)
df_gen["PG > PMAX"] = df_gen["PG"] > df_gen["PMAX"] + tol
violated_idx = df_gen["PG > PMAX"]
print(df_gen.loc[violated_idx, ["PG", "PMAX", "PMIN", "GEN_STATUS", "PG > PMAX"]])

Empty DataFrame
Columns: [PG, PMAX, PMIN, GEN_STATUS, PG > PMAX]
Index: []


In [6]:
from matpower import start_instance
from matpowercaseframes import CaseFrames

import pypglib

In [12]:
m = start_instance()
case_path = pypglib.pglib_opf_case2746wop_k
success = m.eval(
    f"""
    mpopt = mpoption('verbose', 0, 'out.all', 0);
    mpc = loadcase('{case_path}');
    _r1 = runopf(mpc, mpopt);
    _r1.success;
    """
)
if success:
    r1_mpc = m.eval(
        "struct("
        " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus,"
        "  'gen', _r1.gen, 'branch', _r1.branch, 'gencost', _r1.gencost);"
    )

    cf = CaseFrames(r1_mpc)
    cf.infer_numpy()
    cf.gen["PG < PMIN"] = (cf.gen["PG"] < cf.gen["PMIN"] - tol) & (
        cf.gen["GEN_STATUS"] == 1
    )
    cf.gen["PG > PMAX"] = cf.gen["PG"] > cf.gen["PMAX"] + tol
    print(cf.gen[["PG", "PMAX", "PMIN", "GEN_STATUS", "PG < PMIN", "PG > PMAX"]])
m.exit()

               PG     PMAX     PMIN  GEN_STATUS  PG < PMIN  PG > PMAX
1    1.400000e+02  200.000  140.000           1      False      False
2    1.400000e+02  200.000  140.000           1      False      False
3    0.000000e+00  370.000  290.000           0      False      False
4    3.600000e+02  360.000  290.000           1      False      False
5    3.600000e+02  360.000  290.000           1      False      False
6    3.700000e+02  370.000  290.000           1      False      False
7    3.700000e+02  370.000  290.000           1      False      False
8    0.000000e+00  370.000  290.000           0      False      False
9    3.700000e+02  370.000  290.000           1      False      False
10   3.700000e+02  370.000  290.000           1      False      False
11   3.700000e+02  370.000  290.000           1      False      False
12   3.700000e+02  370.000  290.000           1      False      False
13   3.700000e+02  370.000  290.000           1      False      False
14   1.350000e+02  2